In [1]:
import torch

/Users/petersong/Library/Caches/pypoetry/virtualenvs/transformer-8abVYlk8-py3.11/lib/python3.11/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


In [2]:
#Simplified self-attention 코드
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [4]:
#x2에 해당하는 'journey'의 어텐션 스코어 연산
query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [5]:
#이전에 계산한 어텐션 점수를 정규화하여 어텐션 가중치를 계산
atten_weight_2_tmp = attn_scores_2 / attn_scores_2.sum()
print("어텐션 가중치: ", atten_weight_2_tmp)
print("합계: ", atten_weight_2_tmp.sum())

어텐션 가중치:  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
합계:  tensor(1.0000)


In [6]:
#softmax를 활용한 어텐션 정규화
#해당 함수 사용시 큰 값이나 작은 값을 처리할 때 발생 할 수 있는 수치적 불안정이 발생 할 수 있음
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weight_2_naive = softmax_naive(attn_scores_2)
print("어텐션 가중치: ", attn_weight_2_naive)
print("합계: ", attn_weight_2_naive.sum())

어텐션 가중치:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
합계:  tensor(1.)


In [7]:
#파이토치 소프트맥스 사용
#dim을 0을 선택함에 따라 (6,) 형태의 텐서에 대해 소프트맥스 함수를 적용
attn_weight_2 = torch.softmax(attn_scores_2, dim=0)
print("어텐션 가중치: ", attn_weight_2)
print("합계: ", attn_weight_2.sum())

어텐션 가중치:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
합계:  tensor(1.)


In [16]:
#z(2)의 어텐션 스코어 계산
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    print(attn_weight_2[i])
    context_vec_2 += attn_weight_2[i] * x_i
print(context_vec_2)

tensor(0.1385)
tensor(0.2379)
tensor(0.2333)
tensor(0.1240)
tensor(0.1082)
tensor(0.1581)
tensor([0.4419, 0.6515, 0.5683])


In [17]:
#모든 입력 토큰에 대한 어텐션 weights 계산
attn_scores = torch.empty(6,6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [18]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [19]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("두 번째 행의 합계:", row_2_sum)
print("모든 행의 합계:", attn_weights.sum(dim=-1))


두 번째 행의 합계: 1.0
모든 행의 합계: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [20]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [21]:
print("이전의 두 번째 문맥 벡터:", context_vec_2)

이전의 두 번째 문맥 벡터: tensor([0.4419, 0.6515, 0.5683])


In [ ]:
#Sacled dot-product attention
